In [35]:
import pandas as pd

# Load the cleaned energy data
energy_df = pd.read_csv('energy_cleaned.csv')

# Get the last 10 years
last_10_years = energy_df['Year'].max() - 9
energy_last_10_years = energy_df[energy_df['Year'] >= last_10_years]


In [36]:
# Calculate the 10-year average for all energy types (this includes renewables + non-renewables)
avg_all_energy = energy_last_10_years.groupby('Country')['Energy_production'].mean().reset_index()
avg_all_energy = avg_all_energy.rename(columns={'Energy_production': 'avg_production_all_energy'})

# Calculate the 10-year average production for renewables
avg_renewables = energy_last_10_years[energy_last_10_years['Energy_type'] == 'renewables']\
    .groupby('Country')['Energy_production'].mean().reset_index()
avg_renewables = avg_renewables.rename(columns={'Energy_production': 'avg_production_renewables'})

# Calculate the 10-year average production for non-renewables
avg_non_renewables = energy_last_10_years[energy_last_10_years['Energy_type'] == 'non-renewables']\
    .groupby('Country')['Energy_production'].mean().reset_index()
avg_non_renewables = avg_non_renewables.rename(columns={'Energy_production': 'avg_production_non_renewables'})

# Calculate the 10-year average CO2 emissions
avg_co2_emissions = energy_last_10_years.groupby('Country')['CO2_emission'].mean().reset_index()
avg_co2_emissions = avg_co2_emissions.rename(columns={'CO2_emission': 'avg_CO2_emissions'})

# Get the latest year data for GDP and Population
latest_year = energy_df['Year'].max()
latest_data = energy_df[energy_df['Year'] == latest_year].drop_duplicates(subset='Country')
latest_gdp_pop = latest_data[['Country', 'GDP', 'Population']]

In [37]:
# Merge all the data together
merged_data = avg_all_energy\
    .merge(avg_renewables, on='Country', how='left')\
    .merge(avg_non_renewables, on='Country', how='left')\
    .merge(avg_co2_emissions, on='Country', how='left')\
    .merge(latest_gdp_pop, on='Country', how='left')

# Round the necessary columns
merged_data['avg_production_all_energy'] = merged_data['avg_production_all_energy'].round(3)
merged_data['avg_production_renewables'] = merged_data['avg_production_renewables'].round(3)
merged_data['avg_production_non_renewables'] = merged_data['avg_production_non_renewables'].round(3)
merged_data['avg_CO2_emissions'] = merged_data['avg_CO2_emissions'].round(3)
merged_data['GDP'] = merged_data['GDP'].round(2)
merged_data['Population'] = merged_data['Population'].round(2)

In [38]:
# Country codes to full names mapping
country_code_map = {
    'AT': 'Austria', 'BE': 'Belgium', 'BG': 'Bulgaria', 'CH': 'Switzerland',
    'CZ': 'Czech Republic', 'DE': 'Germany', 'DK': 'Denmark', 'EE': 'Estonia',
    'ES': 'Spain', 'FI': 'Finland', 'FR': 'France', 'GB': 'United Kingdom',
    'GR': 'Greece', 'HR': 'Croatia', 'HU': 'Hungary', 'IE': 'Ireland',
    'IT': 'Italy', 'LT': 'Lithuania', 'LU': 'Luxembourg', 'LV': 'Latvia',
    'NL': 'Netherlands', 'NO': 'Norway', 'PL': 'Poland', 'PT': 'Portugal',
    'RO': 'Romania', 'SE': 'Sweden', 'SI': 'Slovenia', 'SK': 'Slovakia'
}

# Reverse the mapping to get country names to codes
country_name_to_code = {v: k for k, v in country_code_map.items()}

# Map country names to codes
merged_data['Country_Code'] = merged_data['Country'].map(country_name_to_code)

# Drop the 'Country' column
merged_data = merged_data.drop(columns=['Country'])


In [39]:
# Load country_predictions.csv
predictions_df = pd.read_csv('country_predictions.csv', index_col=0)

# Reset index to have Country_Code as a column
predictions_df = predictions_df.reset_index()
predictions_df = predictions_df.rename(columns={'index': 'Country_Code'})

# Merge the data on Country_Code
final_df = predictions_df.merge(merged_data, on='Country_Code', how='left')

# Save the final dataframe to country_energy.csv
final_df.to_csv('country_energy.csv', index=False)
